# ENS-GI Digital Twin: Basic Simulation Tutorial

## Introduction to Gut Motility Modeling

This tutorial covers:
1. Creating a basic digital twin
2. Running simulations with different parameters
3. Visualizing neural activity, ICC slow waves, and smooth muscle contraction
4. Parameter sweeps for sensitivity analysis
5. IBS profile comparison

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import sys

sys.path.insert(0, '..')

from ens_gi_core import ENSGIDigitalTwin, IBS_PROFILES

print("✓ ENS-GI Digital Twin imported successfully")
print(f"  Available IBS profiles: {list(IBS_PROFILES.keys())}")

## Part 1: Create Your First Digital Twin

In [ ]:
# Create a digital twin with 10 segments (simplified gut model)
twin = ENSGIDigitalTwin(n_segments=10)

print(f"Created digital twin with:")
print(f"  • {twin.n_segments} gut segments")
print(f"  • {len(twin.network.neurons)} enteric neurons")
print(f"  • 1 ICC (interstitial cell of Cajal) pacemaker")
print(f"  • {twin.n_segments} smooth muscle cells")
print(f"\nCurrent profile: {twin._profile}")

## Part 2: Run a Basic Simulation

In [ ]:
# Run simulation for 2000 ms (2 seconds)
# dt=0.05 ms (20 kHz sampling)
# I_stim: Apply 10 pA stimulus to neuron #5

print("Running simulation...")
result = twin.run(
    duration=2000,
    dt=0.05,
    I_stim={5: 10.0},  # Stimulate neuron 5 with 10 pA
    verbose=True
)

print(f"\nSimulation complete!")
print(f"  Time points: {len(result['time'])}")
print(f"  Voltage shape: {result['voltages'].shape}")
print(f"  Force shape: {result['forces'].shape}")

## Part 3: Visualize the Results

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(4, 1, figsize=(14, 10))

t_plot = result['time'][:2000]  # First 2000 points (~100 ms)

# 1. Neural voltage traces
axes[0].plot(t_plot, result['voltages'][:2000, 0], label='Neuron 0', linewidth=1.5)
axes[0].plot(t_plot, result['voltages'][:2000, 5], label='Neuron 5 (stimulated)', linewidth=1.5)
axes[0].set_ylabel('Voltage (mV)', fontsize=12)
axes[0].set_title('Enteric Neuron Action Potentials', fontsize=14, fontweight='bold')
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=-55, color='red', linestyle='--', alpha=0.5, label='Spike threshold')

# 2. Calcium dynamics
axes[1].plot(t_plot, result['calcium'][:2000, 0], color='orange', linewidth=1.5, label='Neuron 0')
axes[1].plot(t_plot, result['calcium'][:2000, 5], color='red', linewidth=1.5, label='Neuron 5')
axes[1].set_ylabel('[Ca²⁺] (µM)', fontsize=12)
axes[1].set_title('Intracellular Calcium Transients', fontsize=14, fontweight='bold')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

# 3. ICC pacemaker slow wave
t_icc = result['time'][:10000]  # Longer window to see slow wave
axes[2].plot(t_icc, result['icc_currents'][:10000], color='purple', linewidth=2)
axes[2].set_ylabel('ICC Current (pA)', fontsize=12)
axes[2].set_title('ICC Pacemaker Slow Wave (~3 cycles/min)', fontsize=14, fontweight='bold')
axes[2].grid(True, alpha=0.3)

# 4. Smooth muscle force
axes[3].plot(t_icc, result['forces'][:10000, 0], label='Segment 0', linewidth=1.5)
axes[3].plot(t_icc, result['forces'][:10000, 5], label='Segment 5', linewidth=1.5)
axes[3].set_ylabel('Force (normalized)', fontsize=12)
axes[3].set_xlabel('Time (ms)', fontsize=12)
axes[3].set_title('Smooth Muscle Contraction', fontsize=14, fontweight='bold')
axes[3].legend(loc='upper right')
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Part 4: Extract Clinical Biomarkers

In [ ]:
# Extract clinically relevant biomarkers
biomarkers = twin.extract_biomarkers()

print("📊 Clinical Biomarkers:")
print("  " + "-"*50)
for key, value in biomarkers.items():
    print(f"  {key:<30} {value:>10.4f}")
print("  " + "-"*50)

# Generate clinical report
print("\n" + twin.clinical_report())

## Part 5: Parameter Sweep

Explore how changing Na⁺ conductance (g_Na) affects neural excitability.

In [ ]:
# Parameter sweep: g_Na from 80 to 160 mS/cm²
g_Na_values = np.linspace(80, 160, 9)
spike_rates = []
motility_indices = []

print("Running parameter sweep...")
for g_Na in g_Na_values:
    # Create new twin
    sweep_twin = ENSGIDigitalTwin(n_segments=8)
    
    # Modify g_Na for all neurons
    for neuron in sweep_twin.network.neurons:
        neuron.params.g_Na = g_Na
    
    # Run simulation
    sweep_twin.run(1000, dt=0.1, I_stim={3: 10.0}, verbose=False)
    bio = sweep_twin.extract_biomarkers()
    
    spike_rates.append(bio['spike_rate_per_neuron'])
    motility_indices.append(bio['motility_index'])
    print(f"  g_Na = {g_Na:>6.1f} → spike rate = {bio['spike_rate_per_neuron']:.3f} Hz")

# Plot results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Spike rate vs g_Na
ax1.plot(g_Na_values, spike_rates, 'o-', linewidth=2, markersize=8, color='blue')
ax1.set_xlabel('g_Na (mS/cm²)', fontsize=12)
ax1.set_ylabel('Spike Rate (Hz/neuron)', fontsize=12)
ax1.set_title('Neural Excitability vs Na⁺ Conductance', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.axvline(x=120, color='red', linestyle='--', alpha=0.5, label='Healthy baseline')
ax1.legend()

# Motility index vs g_Na
ax2.plot(g_Na_values, motility_indices, 's-', linewidth=2, markersize=8, color='green')
ax2.set_xlabel('g_Na (mS/cm²)', fontsize=12)
ax2.set_ylabel('Motility Index', fontsize=12)
ax2.set_title('Gut Motility vs Na⁺ Conductance', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.axvline(x=120, color='red', linestyle='--', alpha=0.5, label='Healthy baseline')
ax2.legend()

plt.tight_layout()
plt.show()

print("\n📊 Key Findings:")
print("  • Higher g_Na → More excitable neurons → Higher spike rate")
print("  • Higher spike rate → Increased smooth muscle activation → Higher motility")
print("  • IBS-D patients may have elevated g_Na (hyperexcitability)")
print("  • IBS-C patients may have reduced g_Na (hypoexcitability)")

## Part 6: Compare IBS Profiles

In [ ]:
# Compare all IBS subtypes
profiles = ['healthy', 'ibs_d', 'ibs_c', 'ibs_m']
profile_biomarkers = {}

print("Simulating all IBS profiles...\n")
for profile in profiles:
    print(f"Running {profile.upper()}...")
    profile_twin = ENSGIDigitalTwin(n_segments=10)
    profile_twin.apply_profile(profile)
    profile_twin.run(1500, dt=0.05, I_stim={4: 10.0}, verbose=False)
    profile_biomarkers[profile] = profile_twin.extract_biomarkers()

# Create comparison table
print("\n📊 IBS Profile Comparison:")
print("  " + "="*75)
print(f"  {'Profile':<12} {'Motility':<12} {'ICC (cpm)':<12} {'Spike Rate':<15} {'Mean V (mV)'}")
print("  " + "-"*75)

for profile in profiles:
    bio = profile_biomarkers[profile]
    print(f"  {profile.upper():<12} "
          f"{bio['motility_index']:<12.3f} "
          f"{bio['icc_frequency_cpm']:<12.2f} "
          f"{bio['spike_rate_per_neuron']:<15.3f} "
          f"{bio['mean_voltage']:<12.2f}")

print("  " + "="*75)

# Visualize comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

metrics = ['motility_index', 'icc_frequency_cpm', 'spike_rate_per_neuron']
titles = ['Motility Index', 'ICC Frequency (cpm)', 'Spike Rate (Hz/neuron)']
colors = ['#2ecc71', '#e74c3c', '#3498db', '#f39c12']  # Green, Red, Blue, Orange

for idx, (metric, title) in enumerate(zip(metrics, titles)):
    values = [profile_biomarkers[p][metric] for p in profiles]
    axes[idx].bar(profiles, values, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
    axes[idx].set_ylabel(title, fontsize=12)
    axes[idx].set_title(title, fontsize=14, fontweight='bold')
    axes[idx].grid(True, axis='y', alpha=0.3)
    
    # Add value labels on bars
    for i, v in enumerate(values):
        axes[idx].text(i, v, f'{v:.2f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n🔍 Clinical Interpretation:")
print("  • IBS-D (Diarrhea): ↑↑ Motility, ↑ Spike rate → Hyperexcitability")
print("  • IBS-C (Constipation): ↓↓ Motility, ↓ Spike rate → Hypoexcitability")
print("  • IBS-M (Mixed): Variable motility → Unstable dynamics")
print("  • Healthy: Balanced motility and ICC pacing")

## Part 7: Spike Detection and Analysis

In [ ]:
# Analyze spike patterns for one neuron
neuron = twin.network.neurons[5]  # Neuron 5 (the stimulated one)

spike_times = neuron.spike_times
print(f"\n📊 Spike Analysis for Neuron 5:")
print(f"  Total spikes: {len(spike_times)}")

if len(spike_times) > 1:
    # Calculate inter-spike intervals (ISI)
    isis = np.diff(spike_times)
    print(f"  Mean ISI: {np.mean(isis):.2f} ms")
    print(f"  ISI std: {np.std(isis):.2f} ms")
    print(f"  Firing rate: {1000/np.mean(isis):.2f} Hz")
    
    # Plot ISI histogram
    plt.figure(figsize=(10, 5))
    plt.hist(isis, bins=20, color='skyblue', edgecolor='black', alpha=0.7)
    plt.xlabel('Inter-Spike Interval (ms)', fontsize=12)
    plt.ylabel('Count', fontsize=12)
    plt.title('Inter-Spike Interval Distribution', fontsize=14, fontweight='bold')
    plt.axvline(x=np.mean(isis), color='red', linestyle='--', linewidth=2, label=f'Mean = {np.mean(isis):.1f} ms')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("  Not enough spikes for ISI analysis")

## Summary

In this tutorial, you learned:

1. ✅ How to create an ENS-GI Digital Twin
2. ✅ Running simulations with external stimulation
3. ✅ Visualizing neural activity, calcium, ICC, and muscle force
4. ✅ Extracting clinical biomarkers
5. ✅ Performing parameter sweeps (sensitivity analysis)
6. ✅ Comparing IBS profile behaviors
7. ✅ Analyzing spike patterns

### Next Steps:

- Try the **clinical parameter estimation** tutorial (`clinical_workflow.ipynb`)
- Explore **drug trials** tutorial (`virtual_drug_trials.ipynb`)
- Learn about **hardware export** (`hardware_export_tutorial.ipynb`)

### Customize Your Simulations:

```python
# Experiment with different parameters:
twin = ENSGIDigitalTwin(n_segments=20)  # Larger network
twin.network.neurons[0].params.g_Na = 150  # Modify individual neuron
twin.network.icc.omega = 0.012  # Change ICC frequency
twin.run(5000, dt=0.01, I_stim={0: 15, 10: 20})  # Multiple stimuli
```